[View in Colaboratory](https://colab.research.google.com/github/vivek09pathak/ML_Detect_Object/blob/master/Ship_Detector_3-Sep.ipynb)

In [0]:
!git clone https://github.com/vivek09pathak/ML_Detect_Object


In [0]:
cd ML_Detect_Object/

In [0]:
!pip install --user Cython
!pip install --user contextlib2
!pip install --user pillow
!pip install --user lxml
!pip install --user jupyter
!pip install --user matplotlib

In [0]:
!wget -O protobuf.zip https://github.com/google/protobuf/releases/download/v3.0.0/protoc-3.0.0-linux-x86_64.zip
!unzip protobuf.zip

In [0]:
!./bin/protoc object_detection/protos/*.proto --python_out=.

In [0]:
import tensorflow as tf
import numpy as np
import os
import sys
import time
import cv2
import math  
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

In [0]:
# Here are the imports from the object detection module.
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

In [0]:
ls

In [0]:
# Model preparation
MODEL_NAME = './object_detection/models/ssd_mobilenet_v1_ship_15000'
print(MODEL_NAME)

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = MODEL_NAME + '/frozen_inference_graph.pb'
print(PATH_TO_CKPT)
# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('./object_detection/data', 'ship_label_map.pbtxt')
print(PATH_TO_LABELS)
NUM_CLASSES = 1


In [0]:
# Load a (frozen) Tensorflow model into memory.
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(r'./object_detection/models/ssd_mobilenet_v1_ship_15000/frozen_inference_graph.pb', 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')


In [0]:
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

In [0]:
def load_image_into_numpy_array(image):
  (im_width, im_height) = [500,500]
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)


In [0]:

def calculateDistance(x1,y1,x2,y2):  
    dist = math.sqrt((x2 - x1)**2 + (y2 - y1)**2) 
    delta =0.0033 * (dist**2) -1.9918 * dist +372.28
    
    return delta  


In [0]:
def pipeline(image):
  
    with detection_graph.as_default():
          with tf.Session(graph=detection_graph) as sess:
                
                image_np = np.asarray(image)
                image_np_expanded = np.expand_dims(image_np, axis=0)
                image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
                # Each box represents a part of the image where a particular object was detected.
                boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
                # Each score represent how level of confidence for each of the objects.
                # Score is shown on the result image, together with the class label.
                scores = detection_graph.get_tensor_by_name('detection_scores:0')

                classes = detection_graph.get_tensor_by_name('detection_classes:0')
                num_detections = detection_graph.get_tensor_by_name('num_detections:0')
                # Actual detection.
                (boxes, scores, classes, num_detections) = sess.run(
                  [boxes, scores, classes, num_detections],
                  feed_dict={image_tensor: image_np_expanded})
                # Visualization of the results of a detection.
                vis_util.visualize_boxes_and_labels_on_image_array(
                  image_np,
                  np.squeeze(boxes),
                  np.squeeze(classes).astype(np.int32),
                  np.squeeze(scores),
                  category_index,
                  use_normalized_coordinates=True,
                  min_score_thresh=.9,
                  line_thickness=2)
                
                ################### Data analysis ###################
                print("")
                final_score = np.squeeze(scores)  # scores
                r_count = 0  # counting
                r_score = []  # temp score, <class 'numpy.ndarray'>
                final_category = np.array([category_index.get(i) for i in classes[0]]) # category
                r_category = np.array([])  # temp category

                for i in range(100):
                    if scores is None or final_score[i] > 0.7:
                        r_count = r_count + 1
                        r_score = np.append(r_score, final_score[i])
                        r_category = np.append(r_category, final_category[i])

                if r_count > 0:
                    print("Number of bounding boxes: ", r_count)
                    print("")
                else:
                    print("Not Detect")
                    print("")
                for i in range(len(r_score)):  # socre array`s length
                    print("Object Num: {} , Category: {} , Score: {}%".format(i+1, r_category[i]['name'], 100*r_score[i]))
                    print("")
                    final_boxes = np.squeeze(boxes)[i]  # ymin, xmin, ymax, xmax
                    xmin = final_boxes[1]
                    ymin = final_boxes[0]
                    xmax = final_boxes[3]
                    ymax = final_boxes[2]
                    location_x = (xmax+xmin)/2
                    location_y = (ymax+ymin)/2
                    min_location=location_y*100
                    
                    # print("final_boxes [ymin xmin ymax xmax]")
                    # print("final_boxes", final_boxes)
                    if(min_location>35):
                      cv2.putText(image_np,'FAR', (100, 90),  cv2.FONT_HERSHEY_SIMPLEX,1.5, (0,255,0), thickness=2)
                    else:
                      cv2.putText(image_np,'NEAR' , (100, 90),  cv2.FONT_HERSHEY_SIMPLEX,1.5, (255,0,0), thickness=2)
                    print("Location x: {}, y: {}".format(location_x, location_y))
                    print("")
                print("+ " * 30 ) 
               
    return image_np

                



In [0]:
!pip install moviepy

In [0]:

from moviepy.editor import VideoFileClip
vid_output = 'result9_1.mp4'
clip = VideoFileClip("result9.mp4")
vid = clip.fl_image(pipeline)
vid.write_videofile(vid_output, audio=False)

In [0]:

import io# import # import 
import base64
from IPython.display import HTML

video = io.open('result9_1.mp4', 'r+b').read()
encoded = base64.b64encode(video)
HTML(data='''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii')))